# Dependencies and Setup

In [1]:
#!pip install sodapy

In [2]:
# make sure to install these packages before running:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import datetime   # handle date times
import re         # regular expression package
from sqlalchemy import create_engine
import requests
from sodapy import Socrata

In [3]:
# Austin Metro Accident Data

Select only the columns we are intersted in and select only accidents that happend in the Austin Metro Area, betweeen 2016 and 2019

* ID
* Severity
* Start_Time
* End_Time
* Start_Lat
* Start_Lng
* City
* County
* State
* Zipcode
* Country
* Visibility(mi) 
* Weather_Condition
* Precipitation(in)
* Sunrise_Sunset
* Civil_Twilight





In [4]:
# Check if data has been pickled and if so load pickle
if os.path.exists('data/tx_metro_cities.pkl') :
    tx_metro_cities_df = pd.read_pickle('data/tx_metro_cities.pkl',compression='gzip')    
else:
# pickle only the attributes we are interested in, and compress
# pickle is much faster/smaller than csv
    df = pd.read_csv('data/US_Accidents_Dec19.csv',encoding='utf-8')
    columns_of_interest=[ 'ID','Severity','Start_Time','End_Time','Start_Lat', 'Start_Lng', 'City', 
        'County', 'State','Zipcode', 'Country','Visibility(mi)', 
        'Weather_Condition','Precipitation(in)','Sunrise_Sunset','Civil_Twilight'
    ]
    condition=df['State'] =='TX'
    txDF=df[condition][columns_of_interest]
    #create df for selected cities 
    tx_metro_cities =  ['Austin', 'Round Rock', 'Cedar Park',
                 'San Marcos', 'Georgetown', 'Pflugerville',
                 'Hutto', 'Leander'  ]
    condition = txDF['City'].isin(tx_metro_cities)
    tx_metro_cities_df = txDF[condition]    
    tx_metro_cities_df.to_pickle("data/tx_metro_cities.pkl",compression='gzip')
    



In [5]:
# add_datepart from fastai
# Date Feature Engineering
# will split data colum in to corresponding 'Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear'
# 'Hour', 'Minute'
def add_datepart(df, fldname, drop=True,time=False):
    fld = df[fldname]
    attributes = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear']
    if time: attributes = attributes + ['Hour', 'Minute']
    #, 'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if not np.issubdtype(fld.dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    for n in attributes:
        df[targ_pre+n] = getattr(fld.dt,n.lower())
    df[targ_pre+'Elapsed'] = fld.astype(np.int64) // 10**9
    if drop: df.drop(fldname, axis=1, inplace=True)
        
add_datepart(tx_metro_cities_df,'Start_Time',False,True )        

In [6]:
# use name for DOW
def dow(df):
    days=["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
    return days[df]

tx_metro_cities_df['Start_TimeDayofweek'] =tx_metro_cities_df['Start_TimeDayofweek'].apply(dow)



In [7]:
condition = tx_metro_cities_df['Start_TimeYear'].isin([2017,2018])
tx_metro_cities_df = tx_metro_cities_df[condition]  

In [8]:
print(f"Number of Columns : {len(tx_metro_cities_df.columns)}")
print(f"Number of Rows : {len(tx_metro_cities_df)}")
print(tx_metro_cities_df.columns)

tx_metro_cities_df.head()


Number of Columns : 25
Number of Rows : 33445
Index(['ID', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng',
       'City', 'County', 'State', 'Zipcode', 'Country', 'Visibility(mi)',
       'Weather_Condition', 'Precipitation(in)', 'Sunrise_Sunset',
       'Civil_Twilight', 'Start_TimeYear', 'Start_TimeMonth', 'Start_TimeWeek',
       'Start_TimeDay', 'Start_TimeDayofweek', 'Start_TimeDayofyear',
       'Start_TimeHour', 'Start_TimeMinute', 'Start_TimeElapsed'],
      dtype='object')


,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,City,County,State,Zipcode,...,Civil_Twilight,Start_TimeYear,Start_TimeMonth,Start_TimeWeek,Start_TimeDay,Start_TimeDayofweek,Start_TimeDayofyear,Start_TimeHour,Start_TimeMinute,Start_TimeElapsed
267710,A-267712,2,2017-01-02 14:40:12,2017-01-02 15:39:53,30.393579,-97.745979,Austin,Travis,TX,78759-5803,...,Day,2017,1,1,2,Monday,2,14,40,1483368012
267711,A-267713,2,2017-01-02 14:28:48,2017-01-02 15:28:36,30.256216,-97.722038,Austin,Travis,TX,78702-4564,...,Day,2017,1,1,2,Monday,2,14,28,1483367328
267712,A-267714,2,2017-01-02 14:04:17,2017-01-02 15:05:00,30.418287,-97.700874,Austin,Travis,TX,78758-2412,...,Day,2017,1,1,2,Monday,2,14,4,1483365857
267713,A-267715,2,2017-01-02 14:29:13,2017-01-02 15:35:00,30.237938,-97.696167,Austin,Travis,TX,78741,...,Day,2017,1,1,2,Monday,2,14,29,1483367353
267714,A-267716,2,2017-01-02 14:40:51,2017-01-02 15:40:23,30.364882,-97.695992,Austin,Travis,TX,78753-4107,...,Day,2017,1,1,2,Monday,2,14,40,1483368051


# Austin Fatality Data

In [9]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.austintexas.gov", None)


# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.

results = client.get("ergh-7g8p", limit=2000)
results_df_2012 =pd.DataFrame.from_records(results)

results = client.get("vggi-9ddh", limit=2000)
results_df_2013 =pd.DataFrame.from_records(results)

results = client.get("gm9p-snyb", limit=2000)
results_df_2014=pd.DataFrame.from_records(results)

results = client.get("p658-umsa", limit=2000)
results_df_2015=pd.DataFrame.from_records(results)

results = client.get("tiqb-wv3c", limit=2000)
results_df_2016=pd.DataFrame.from_records(results)

results = client.get("ijds-pcyq", limit=2000)
results_df_2017=pd.DataFrame.from_records(results)

results = client.get("9jd4-zjmx", limit=2000)
results_df_2018 = pd.DataFrame.from_records(results)



# Rename columns to consistent name between different years

In [10]:
# #x_coord
# results_df_2016.rename(columns = {'coord_x':'x_coord'}, inplace = True)

# #dl_status_incident
# results_df_2016.rename(columns = {'dl_status':'dl_status_incident'}, inplace = True)

# #killed_driver_pass
# results_df_2016.rename(columns = {'victim':'killed_driver_pass'}, inplace = True)

# #restraint_type
# results_df_2016.rename(columns = {'restraint_or_helmet':'restraint_type'}, inplace = True)
results_df_2017.rename(columns = {'restraint_helmet':'restraint_type'}, inplace = True)


#ftsra
results_df_2017.rename(columns = {'failure_to_stop_and_render_aid':'ftsra'}, inplace = True)
results_df_2018.rename(columns = {'failure_to_stop_and_render_aid':'ftsra'}, inplace = True)

In [11]:
#concatenate 2016-2018 fatality data
results_project= pd.concat([ results_df_2017, results_df_2018])



# Data type changes

In [12]:
results_project = results_project.astype({ "x_coord":float, "y_coord": float})
results_project['date'] =  pd.to_datetime(results_project['date'], infer_datetime_format=True)
results_project['year'] =  results_project['date'].dt.year


In [13]:
print(f"Number of Columns : {len(results_project.columns)}")
print(f"Number of Rows : {len(results_project)}")
print(results_project.columns)

results_project.head()


Number of Columns : 25
Number of Rows : 144
Index(['type', 'fatal_crash_number', 'number_of_fatalities', 'case_number',
       'location', 'area', 'date', 'month', 'day', 'hour', 'time', 'related',
       'charge', 'killed_driver_pass', 'speeding',
       'ran_red_light_or_stop_sign', 'dl_status_incident',
       'suspected_impairment', 'restraint_type', 'type_of_road', 'ftsra',
       'x_coord', 'y_coord', 'case_status', 'year'],
      dtype='object')


,type,fatal_crash_number,number_of_fatalities,case_number,location,area,date,month,day,hour,...,ran_red_light_or_stop_sign,dl_status_incident,suspected_impairment,restraint_type,type_of_road,ftsra,x_coord,y_coord,case_status,year
0,Motor Vehicle,1,1,17-0030106,8200 blk N. Lamar Blvd,ED,2017-01-03,Jan,Tue,1,...,N,okay,DRIVER,seatbelt worn,high use roadway,N,-97.709153,30.350809,NaN,2017
1,Bicycle,2,1,17-0080877,Research Blvd SVRD NB/Riata Trace Pkwy,AD,2017-01-08,Jan,Sun,15,...,Y,suspended,NONE,helmet worn,other highway,N,-97.753497,30.426752,NaN,2017
2,Motor Vehicle,3,1,17-0071308,E US Hwy 290 WB Svrd/Johnny Morris,CH,2017-01-07,Jan,Sat,21,...,N,no DL,DRIVER,unknown,other highway,Y,-97.623275,30.331230,NaN,2017
3,Motor Vehicle,4,1,17-0150225,13500 N US 183 NB,AD,2017-01-15,Jan,Sun,2,...,N,no DL,BOTH Drivers,no seatbelt,other highway,N,-97.790548,30.449598,NaN,2017
4,Pedestrian,5,1,17-0110340,5900 blk N IH35,ID,2017-01-11,Jan,Wed,6,...,N,okay,PED,n/a,IH35,N,-97.706646,30.320762,NaN,2017


# Save to SQLlite


In [14]:
engine = create_engine('sqlite:///data/AUSaccidents.db', echo=False)
tx_metro_cities_df.to_sql('austinAccidents',con=engine,if_exists='replace')


In [15]:
results_project.to_sql('austinFatalities',con=engine,if_exists='replace')
